In [12]:
import polars as pl
import pyodbc

In [13]:
import functions.common as fn

### Definición de Rutas

In [14]:
cierre = "202503"
cierre_l = "Mar25"

# Carpetas
path = "E:/Users/jhernandezr/DAR/garantias/reporte/fotos/"
wd_data_raw = path + "data/raw/"
wd_data_external = path + "data/external/"
wd_data_processed_dwh = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/DWH/csv/"
wd_data_processed_dwh = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/Fotos/csv/"

# Inputs
fl_catalogos = wd_data_external + f'Catálogos_{cierre_l}'
fl_cohortes_1 = wd_data_raw + f'Cohortes_{cierre}_1.mdb'
fl_cohortes_2 = wd_data_raw + f'Cohortes_{cierre}_2.mdb'
fl_cohortes_3 = wd_data_raw + f'Cohortes_{cierre}_3.mdb'
fl_cohortes_4 = wd_data_raw + f'Cohortes_{cierre}_4.mdb'
fl_cohortes_5 = wd_data_raw + f'Cohortes_{cierre}_5.mdb'

# Outputs


In [ ]:
conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + fl_catalogos + '.accdb'
conn = pyodbc.connect(conn_str)
tipo_credito = pl.read_database(query='SELECT * FROM [TIPO_CREDITO]', connection=conn, schema_overrides={'Tipo_Credito_ID': pl.Int32})
conn.close()


<function Connection.close()>

### 04 Carga BD

In [16]:
schema_cohortes = {
        'BUCKET': pl.Int16,
        'CONREC_CLAVE': pl.Int32,
        'DESC_INDICADOR': pl.String,
        'Describe_Desrec': pl.String,
        'ESTADO_ID': pl.Int32,
        'ESTRATO_ID': pl.Int16,
        'FECHA_APERTURA': pl.Datetime,
        'FECHA_PRIMER_INCUMPLIMIENTO': pl.Datetime,
        'FECHA_REGISTRO_ALTA': pl.Datetime,
        'INDICADOR_ID': pl.Int32,
        'INTERMEDIARIO_ID': pl.String,
        'NOMBRE_EMPRESA': pl.String,
        'NUMERO_CREDITO': pl.String,
        'PLAZO': pl.Int32,
        'PLAZO_DIAS': pl.Int32,
        'PORCENTAJE_COMISION_GARANTIA': pl.Float32,
        'PORCENTAJE_GARANTIZADO': pl.Float32,
        'PROGRAMA_ID': pl.Int32,
        'PROGRAMA_ORIGINAL': pl.Int32,
        'RAZON_SOCIAL': pl.String,
        'RFC_EMPRESA': pl.String,
        'SECTOR_ID': pl.Int16,
        'TASA_ID': pl.Int16,
        'TIPO_CREDITO_ID': pl.Int32,
        'TIPO_GARANTIA_ID': pl.Int32,
        'VALOR_TASA_INTERES': pl.Float32,
        'MONTO_CREDITO_MN (SUMA)': pl.Float64
}

In [17]:
def read_access(file, table, schema):
    conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + file
    conn = pyodbc.connect(conn_str)
    df = pl.read_database(query=f"SELECT * FROM {table}", connection=conn, schema_overrides=schema)
    conn.close()
    return df


In [31]:
def import_bd_dwh():
    cohortes_1 = read_access(fl_cohortes_1, "DATOS", schema_cohortes)
    cohortes_2 = read_access(fl_cohortes_2, "DATOS", schema_cohortes)
    cohortes_3 = read_access(fl_cohortes_3, "DATOS", schema_cohortes)
    cohortes_4 = read_access(fl_cohortes_4, "DATOS", schema_cohortes)
    cohortes_5 = read_access(fl_cohortes_5, "DATOS", schema_cohortes)

    db_dwh = pl.concat([
                        cohortes_1, 
                        cohortes_2, 
                        cohortes_3, 
                        cohortes_4, 
                        cohortes_5
                        ], rechunk=True, how="diagonal")
    return db_dwh

db_dwh = import_bd_dwh()

In [ ]:
def genera_dwh_nr_r(db_dwh):
    db_dwh = (db_dwh
            .join(tipo_credito, 
                    how="left", 
                    left_on="TIPO_CREDITO_ID", 
                    right_on="Tipo_Credito_ID"
                )
            )
    
    db_dwh = db_dwh.select(
        pl.col("BUCKET"),
        pl.col("DESC_INDICADOR").alias("Producto"),
        pl.col("ESTADO_ID"),
        pl.col("ESTRATO_ID"),
        pl.col("FECHA_APERTURA").alias("Fecha de Apertura"),
        pl.col("FECHA_PRIMER_INCUMPLIMIENTO"),
        pl.col("FECHA_REGISTRO_ALTA").alias("Fecha Registro Alta"),
        pl.col("INDICADOR_ID").alias("Producto ID"),
        pl.col("INTERMEDIARIO_ID"),
        pl.col("NOMBRE_EMPRESA").alias("Empresa / Acreditado (Descripción)"),
        pl.col("NUMERO_CREDITO"),
        pl.col("PLAZO"),
        pl.col("PLAZO_DIAS").alias("Plazo Días"),
        pl.col("PORCENTAJE_COMISION_GARANTIA").alias("Porcentaje de Comisión Garantia"),
        pl.col("PORCENTAJE_GARANTIZADO").alias("Porcentaje Garantizado"),
        pl.col("PROGRAMA_ID"),
        pl.col("PROGRAMA_ORIGINAL"),
        pl.col("RAZON_SOCIAL").alias("Razón Social (Intermediario)"),
        pl.col("RFC_EMPRESA").alias("RFC Empresa / Acreditado"),
        pl.col("SECTOR_ID"),
        pl.col("TASA_ID"),
        pl.col("TIPO_CREDITO_ID"),
        pl.col("TIPO_GARANTIA_ID"),
        pl.col("VALOR_TASA_INTERES"),
        pl.col("MONTO_CREDITO_MN (SUMA)").alias("Monto _Credito_Mn"),
        pl.col("CONREC_CLAVE"),
        pl.col("Describe_Desrec"),
        pl.col("NR_R")
    )

    db_dwh_r = db_dwh.filter(pl.col("NR_R")=="R")
    db_dwh_nr = db_dwh.filter(pl.col("NR_R")=="NR")

    return db_dwh_r, db_dwh_nr

db_dwh_r, db_dwh_nr = genera_dwh_nr_r(db_dwh)

In [ ]:
def generate_fvto(df):
    result = (df
        .with_columns(
            (pl.col("Fecha de Apertura") +
            pl.duration(days=(365 * pl.col("PLAZO") / 12).round()) + 
            pl.duration(days=(pl.col("Plazo Días").fill_null(0)))
            )
            .alias("test")))
    return result

db_dwh_r = generate_fvto(db_dwh_r)
db_dwh_nr = genera_dwh_nr_r(db_dwh_nr)

In [ ]:
def genera_tpro_clave(df):
    result = df.with_columns(
        pl.when((pl.col("PROGRAMA_ID")>=32000)&(pl.col("PROGRAMA_ID")<=32100))
        .then(pl.col("PROGRAMA_ID"))
        .when((pl.col("PROGRAMA_ID")==3976)&(pl.col("PROGRAMA_ORIGINAL")==31415))
        .then(pl.col("PROGRAMA_ID"))
        .when((pl.col("PROGRAMA_ID")==33366)&(pl.col("PROGRAMA_ORIGINAL")==33842))
        .then(pl.col("PROGRAMA_ID"))
        .when((pl.col("PROGRAMA_ID").is_in([3536, 3537, 3539, 3542,3544, 3545, 3546,3547,3548,3549,3550, 3553, 3555, 3558,3559, 3560, 3564,3566]))&(pl.col("PROGRAMA_ORIGINAL")==3200))
        .then(pl.col("PROGRAMA_ID"))
        .when(pl.col("PROGRAMA_ORIGINAL")==3999)
        .then("PROGRAMA_ID")
        .otherwise(pl.col("PROGRAMA_ORIGINAL")).alias("TPRO_CLAVE")
    )
    return result

db_dwh_r = genera_tpro_clave(db_dwh_r)
db_dwh_nr = genera_tpro_clave(db_dwh_nr)

In [ ]:
def genera_tipo_persona(df):
    result = df.with_columns(
        pl.when(pl.col("RFC Empresa / Acreditado").str.slice(3, length=1) == "-")
        .then(pl.lit("M"))
        .otherwise(pl.lit("F")).alias("Tipo_Persona")
    )
    return result

db_dwh_r = genera_tipo_persona(db_dwh_r)
db_dwh_nr = genera_tipo_persona(db_dwh_nr)

In [ ]:
def genera_nombrev1(df):
    result = (
        df.with_columns(
            pl.col("Empresa / Acreditado (Descripción)").str.replace("'", "")
                .alias("Nombre_v1")
        )
    ) 
    return result

db_dwh_r = genera_nombrev1(db_dwh_r)
db_dwh_nr = genera_nombrev1(db_dwh_nr)

### 05 Genera prefotos

### 06 Sube Saldos y MGI

### 07 Genera Saldos